In [35]:
from metavision_core.event_io import EventsIterator, load_events
import tonic
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import random
from IPython.utils import io
# import sdl2.ext

import models
import utils

In [36]:
# %matplotlib inline
%matplotlib ipympl
# %matplotlib tk
plt.rc('animation', html='jshtml')
plt.rcParams["image.cmap"] = "bone"

In [45]:
in_data = tonic.datasets.NMNIST(save_to="../data", train=True) # (34, 34, 2) 3
# in_data = tonic.datasets.POKERDVS(save_to="../data",train=True) # (35, 35, 2) 3
# in_data = tonic.datasets.DVSGesture(save_to="../data",train=True) # (128, 128, 2) 2
# in_data = tonic.datasets.ASLDVS(save_to="../data", train=True) # (240, 180, 2) 3

in_data.sensor_size

  0%|          | 0/691455012 [00:00<?, ?it/s]

Extracting ../data/DVSGesture/ibmGestureTest.tar.gz to ../data/DVSGesture


(128, 128, 2)

In [10]:
noise = tonic.transforms.UniformNoise(
    sensor_size=in_data.sensor_size[:-1]+(1,),
    n=int(4172),
)
# to_frame = tonic.transforms.ToFrame(in_data.sensor_size[:-1]+(1,), time_window=100)
to_frame = tonic.transforms.ToFrame(in_data.sensor_size[:-1]+(1,), n_time_bins=100)

In [11]:
events, label = in_data[random.randint(0, len(in_data)-1)]
events = events.squeeze()
events = np.array(list(filter(lambda ev: ev[3]==0, events)))
# print(events)
print(label)

true_frames = torch.tensor(to_frame(events)).float()#.cuda(0)
in_frames = torch.tensor(to_frame(noise(events))).float()#.cuda(0)

0


In [12]:
# ani.save("./img/nmnist-9.gif", writer="imagemagick")

In [13]:
net = models.Fast1()#.cuda(0)
with torch.inference_mode():
    out_frames = net(in_frames)

In [14]:
in_frames = utils.frame_merge(in_frames,5)
out_frames = utils.frame_merge(out_frames,5)
true_frames = utils.frame_merge(true_frames,5)

In [15]:
loss=nn.L1Loss()
print("noisy loss:"+ str(loss(in_frames,true_frames)))
print("denoised loss:"+ str(loss(out_frames,true_frames)))

noisy loss:tensor(0.1804)
denoised loss:tensor(0.2282)


In [16]:
in_frames = in_frames.cpu()
out_frames = out_frames.cpu()
true_frames = true_frames.cpu()

In [17]:
with io.capture_output() as captured:
    anim = tonic.utils.plot_animation(np.concatenate((in_frames,out_frames,true_frames),axis=3), 30)
    # anim = tonic.utils.plot_animation(np.concatenate((in_frames,out_frames),axis=3), 300)
display(anim)

In [18]:
# anim.save("./img/dataline3.gif", fps=300)